### 将测试数据图片生成为应用多个interpolation的目录, 然后区分目录查看是否预测有区分

In [1]:
import cv2
import numpy as np
import os
CROP_SIZE = 448

def crop_img_and_remove_black_edge(img, true_width_left: int,
                                   true_width_right: int):
    r, c = img.shape[:2]
    true_width = true_width_right - true_width_left + 1
    if true_width < 1:
        raise ValueError("true width wrong!")
    row_start = 0
    if true_width > r:
        raise ValueError(f"true width can not be cuted r{r}c{c}t{true_width}")
    else:
        col_start = true_width_left
        size = true_width
    xshift = col_start
    yshift = row_start
    return img[row_start:row_start + size, col_start:col_start +
               size], xshift, yshift

In [2]:
def get_image_actual_width_index_range_mem(imagebuff=None):
    """
        根据已经读入内存的图片的矩阵获取图像有效宽度, 输入参数类型为numpy.ndarray
        返回tuple(left, right) 代表有效列的索引
    """
    if not imagebuff.any() or not isinstance(imagebuff, np.ndarray):
        raise ValueError("param is not np.ndarray")
    if len(imagebuff.shape) != 2 and len(imagebuff.shape) != 3:
        raise ValueError(
            "image must in IMREAD_GRAYSCALE or IMREAD_COLOR(height, row, gray)!"
        )

    gray_img = imagebuff.sum(axis=2) if len(imagebuff) == 3 else imagebuff

    gray_img_t = gray_img.transpose()

    left = 0
    right = gray_img_t.shape[0] - 1
    for col in range(gray_img_t.shape[0]):
        column = gray_img_t[col]
        column.sort()
        top20 = column[-20:]
        if top20.mean() > 10:
            left = col
            break
    for col in range(gray_img_t.shape[0] - 1, -1, -1):
        column = gray_img_t[col]
        column.sort()
        top20 = column[-20:]
        if top20.mean() > 10:
            right = col
            break
    return (left, right)

In [3]:
def correct_image_size(img_array, zoom=False):
    """
        crop+resize. 返回处理好的图片并返回X轴偏移以及缩放比例. 
    """
    #print(f"correct_image_size: array shape is {img_array.shape}, type({type(img_array)})")
    interpolations = [cv2.INTER_AREA,  cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4 ]
    imgs = []
    if not img_array.any() or not isinstance(img_array, np.ndarray):
        raise ValueError("param is not np.ndarray")
    if len(img_array.shape) != 2 and len(img_array.shape) != 3:
        raise ValueError(
            "image must in IMREAD_GRAYSCALE or IMREAD_COLOR(height, row, gray)!"
        )
    true_width = get_image_actual_width_index_range_mem(img_array.copy())
    img_croped, xshift, _ = crop_img_and_remove_black_edge(
        img_array, true_width[0], true_width[1])
    if zoom:
        for inter in interpolations:
            print("interpolation:", inter)
            img_array_new = cv2.resize(img_croped,
                                       dsize=(CROP_SIZE, CROP_SIZE),
                                       interpolation=inter)
            imgs.append(img_array_new)
            break
    else:
        img_array_new = img_croped
        imgs.append(img_array_new)
    ratio = img_array_new.shape[0] / img_croped.shape[0]
    return imgs, xshift, ratio

In [4]:
def read_image_and_correct_it(imagefile=None, zoom=False):
    """
    读取图片, 如果图片不是448*448的那么进行crop, resize
    注意: 此函数调用后需要配套修改json的坐标
    """
    #print(f"read_image_and_correct_it: file is {imagefile}")
    if not imagefile:
        raise ValueError("imagefile is None")
    if not os.path.isfile:
        msg = f"imagefile {imagefile} not exist"
        raise ValueError(msg)
    img = cv2.imread(imagefile, cv2.IMREAD_GRAYSCALE)
    if not img.size:
        raise ValueError("imagefile read None")
    #print(f"read_image_and_correct_it: array shape is {img.shape}, type({type(img)})")
    xshift = 0
    ratio = 1
    if img.shape[0:2] == (CROP_SIZE, CROP_SIZE):
        new_img = img
    else:
        new_img, xshift, ratio = correct_image_size(img, zoom)

    return (new_img, xshift, ratio)

In [10]:
def generate_interpolation_image(dirr=r"F:\workspace\for_clahe\1.3.6.1.4.1.52026.1.1.1_frms"):
    files = os.listdir(dirr)
    out_dir = dirr+"AREA"
    os.makedirs(out_dir, exist_ok=True)
    for ct, inf in enumerate(files):
        infile = os.path.join(dirr, inf)
        if "png" not in inf:
            continue
        #outf = inf.split(".png")[0]+"clahe.png"
        outf = os.path.join(out_dir, inf)
        print(outf)
        imgs, x, r = read_image_and_correct_it(infile, True)
        #print(type(imgs), imgs.shape)
        #combine_img = merge_multi_image_2_one([imgs, apply_clahe(imgs)])
        cv2.imwrite(outf, imgs[0])
        break

In [11]:
testdir=r"F:\workspace\testSet5.0_raw\03_04_XX_Plaque_L_01_frms"
generate_interpolation_image(testdir)

F:\workspace\testSet5.0_raw\03_04_XX_Plaque_L_01_frmsAREA\frm-0001.png
interpolation: 3
